# Jupyter Notebook to train a model

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tf_explain.callbacks.grad_cam import GradCAMCallback

# use random seed to reproduce results
np.random.seed(42)
tf.random.set_seed(42)

2021-12-06 19:04:34.001788: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


lists possible devices (CPU, GPU), used to check if GPU is recognized/exists

In [2]:
tf.config.get_visible_devices()

2021-12-06 19:04:38.272739: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2021-12-06 19:04:38.273599: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-06 19:04:38.330805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 19:04:38.330990: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.8225GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-12-06 19:04:38.331011: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-12-06 19:04:38.332986: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-12-06 19:04:38.333125: I tensorflow/stream_executor/pl

## Methods

### used to save a trained model as a json file and its weights as a h5 file

In [3]:
def save_model(model, model_name):
    my_model = model.to_json()
    with open(f'./saved_models/{model_name}.json', "w") as file:
        file.write(my_model)
    # serialize weights to HDF5
    model.save_weights(f'./saved_models/{model_name}_weights.h5')

### used to build the base model using predefined architectures
currently: vgg16, xception, resnet

In [4]:
def build_base_model(architecture, weights):
    input = tf.keras.Input(shape=(224, 224, 3))
    if architecture == 'vgg16':
        return tf.keras.applications.vgg16.VGG16(weights=weights, include_top=False, input_tensor=input)
    if architecture == 'xception':
        return tf.keras.applications.xception.Xception(weights=weights, include_top=False, input_tensor=input)
    if architecture == 'resnet':
        return tf.keras.applications.resnet.ResNet50(weights=weights, include_top=False, input_tensor=input)

### gets base model as input and builds a new top layer and returns the model with custom top layers

In [5]:
def build_model(base_model):
    flat = keras.layers.Flatten(name='flatten')(base_model.output)
    dense_1 = keras.layers.Dense(1000)(flat)
    dropout = keras.layers.Dropout(0.25)(dense_1)
    batch = keras.layers.BatchNormalization()(dropout)
    output = keras.layers.Dense(1, activation='sigmoid')(batch)
    return tf.keras.Model(base_model.input, output)

### gets a model as input and returns a model compiled with the adam optimizer

In [6]:
def compile_model(model, alpha, beta1, beta2, metrics):
    optimizer = keras.optimizers.Adam(learning_rate=alpha, beta_1=beta1, beta_2=beta2)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=metrics)
    return model

### gets a model as input and trains it on the data-set with the defined callbacks and epochs

In [7]:
def train_model(model, train_set, validation_set, epochs, callback):
    return model.fit(train_set,
                     validation_data=validation_set,
                     epochs=epochs,
                     callbacks=callback)

### gets a model as input and changes its layers trainable attribute

In [8]:
def set_layers_trainable(trainable, input_model):
    for layer in input_model.layers:
        layer.trainable = trainable

## Data preprocessing

preprocessing of the images applied when loading image data set from disk with tensorflows flow_from_directory


In [9]:
image_gen = keras.preprocessing.image.ImageDataGenerator(rotation_range=20,  # rotate the image 20 degrees
                                                         width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         rescale=1 / 255,  # Rescale the image by normalzing it.
                                                         shear_range=0.15,
                                                         # Shear means cutting away part of the image (max 20%)
                                                         zoom_range=0.15,  # Zoom in by 15% max
                                                         horizontal_flip=True,  # Allow horizontal flipping
                                                         fill_mode='nearest'
                                                         # Fill in missing pixels with the nearest filled value
                                                         )

path to the data set

In [10]:
train_data_path = './data/data_heavily_reduced/data_balanced/train'  #local notebook
test_data_path = './data/data_heavily_reduced/data_balanced/test'  #local notebook
validation_data_path = './data/data_heavily_reduced/data_balanced/val'  #local notebook

generate training set by loading the images from their directories with flow_from_directory
important: the folder structure has to match! i.e {train} -> {ok,def}
at the "same time" the data augmentation is applied on the images through the ImageDataGenerator

In [11]:
train_image_gen = image_gen.flow_from_directory(train_data_path,
                                                target_size=(224, 224),
                                                class_mode='binary')

Found 7999 images belonging to 2 classes.


In [12]:
valid_image_gen = image_gen.flow_from_directory(validation_data_path,
                                                target_size=(224, 224),
                                                class_mode='binary')

Found 999 images belonging to 2 classes.


In [13]:
test_image_gen = image_gen.flow_from_directory(test_data_path,
                                               target_size=(224, 224),
                                               class_mode='binary')

Found 1001 images belonging to 2 classes.


# Training of the model

### Pre Training
* build base model with vgg16 architecture and pretrained with imagenet weights
* add custom layers on base model
* compile model with adam optimizer
* freeze layers of pretrained vgg16 base model to not destroy imagenet weights
* train model

In [15]:
vgg16 = build_base_model('vgg16', 'imagenet')
model = build_model(vgg16)
model = compile_model(model, 0.0001, 0.9, 0.999, ['accuracy', 'Recall', 'Precision', 'AUC'])
set_layers_trainable(False, vgg16)
history = train_model(model, train_image_gen, test_image_gen, 5, [])

2021-12-06 19:04:56.462578: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-06 19:04:56.462984: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 19:04:56.463114: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.8225GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-12-06 19:04:56.463145: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library li

Epoch 1/5


2021-12-06 19:04:59.071855: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-12-06 19:04:59.368990: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-12-06 19:04:59.863236: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-12-06 19:04:59.896238: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-12-06 19:05:11.953312: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


164/250 [==================>...........] - ETA: 30s - loss: 0.4225 - accuracy: 0.8313 - recall: 0.8376 - precision: 0.8333 - auc: 0.9039

2021-12-06 19:06:24.883311: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.01GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


250/250 [==============================] - 146s 524ms/step - loss: 0.3642 - accuracy: 0.8554 - recall: 0.8617 - precision: 0.8548 - auc: 0.9251 - val_loss: 0.1144 - val_accuracy: 0.9590 - val_recall: 1.0000 - val_precision: 0.9244 - val_auc: 0.9968
Epoch 2/5
250/250 [==============================] - 115s 459ms/step - loss: 0.1277 - accuracy: 0.9497 - recall: 0.9538 - precision: 0.9443 - auc: 0.9895 - val_loss: 0.0482 - val_accuracy: 0.9850 - val_recall: 0.9860 - val_precision: 0.9841 - val_auc: 0.9987
Epoch 3/5
250/250 [==============================] - 141s 560ms/step - loss: 0.0888 - accuracy: 0.9674 - recall: 0.9701 - precision: 0.9662 - auc: 0.9945 - val_loss: 0.1351 - val_accuracy: 0.9441 - val_recall: 1.0000 - val_precision: 0.8995 - val_auc: 0.9987
Epoch 4/5
250/250 [==============================] - 124s 497ms/step - loss: 0.0961 - accuracy: 0.9646 - recall: 0.9658 - precision: 0.9637 - auc: 0.9937 - val_loss: 0.1606 - val_accuracy: 0.9341 - val_recall: 0.9960 - val_precision:

### Full Training
* un freeze layers of pretrained vgg16 base model
* define callbacks
    * tensorboard callback to generate reports which can be viewed in tensorboard
    * early stopping callback to stop training after monitored metric has not changed after, by the patience defined, epochs
* compile model with adam optimizer
* train model

In [15]:
set_layers_trainable(True, vgg16)

#### Callbacks

In [18]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=5, write_graph=True, write_images=True)
custom_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

2021-12-06 18:45:53.414652: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-12-06 18:45:53.414674: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-12-06 18:45:53.414694: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 1 GPUs
2021-12-06 18:45:53.415241: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcupti.so.10.1
2021-12-06 18:45:53.515541: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2021-12-06 18:45:53.515837: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [19]:
model = compile_model(model, 0.0001, 0.9, 0.999, ['accuracy', 'Recall', 'Precision', 'AUC'])
history = train_model(model, train_image_gen, test_image_gen, 100,
                      [custom_early_stopping, tensorboard_callback, grad_cam_callback])

Epoch 1/100
  1/250 [..............................] - ETA: 7:30 - loss: 0.0457 - accuracy: 0.9688 - recall: 1.0000 - precision: 0.9231 - auc: 1.0000

2021-12-06 18:52:25.843420: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-12-06 18:52:25.843451: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-12-06 18:52:25.844023: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES


  2/250 [..............................] - ETA: 1:49 - loss: 0.0867 - accuracy: 0.9688 - recall: 0.9839 - precision: 0.9454 - auc: 0.9905

2021-12-06 18:52:26.236676: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-12-06 18:52:26.240232: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-12-06 18:52:26.241644: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-12-06 18:52:26.243437: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2021_12_06_18_52_26
2021-12-06 18:52:26.244167: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./logs/train/plugins/profile/2021_12_06_18_52_26/pop-os.trace.json.gz
2021-12-06 18:52:26.259187: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2021_12_06_18_52_26
2021-12-06 18:52:26.262823: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped

250/250 [==============================] - 98s 385ms/step - loss: 0.0855 - accuracy: 0.9699 - recall: 0.9728 - precision: 0.9675 - auc: 0.9941 - val_loss: 0.3981 - val_accuracy: 0.8781 - val_recall: 1.0000 - val_precision: 0.8042 - val_auc: 0.9796


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type tensorflow.python.framework.ops.EagerTensor).

# Evaluation of the model on the test set

In [36]:
loss, accuracy, recall, precision, auc = model.evaluate(test_image_gen)

32/32 [==============================] - 8s 241ms/step - loss: 0.0188 - accuracy: 0.9950 - recall: 0.9960 - precision: 0.9940 - auc: 0.9998


In [37]:
# F1 score
f1 = 2 * ((precision * recall) / (precision + recall))

print(f"loss: {loss}, \n"
      f"accuracy: {accuracy}, \n"
      f"recall: {recall}, \n"
      f"precision: {precision}, \n"
      f"auc: {auc}, \n"
      f"F1: {f1}")

loss: 0.018838077783584595, 
accuracy: 0.9950050115585327, 
recall: 0.9960079789161682, 
precision: 0.9940239191055298, 
auc: 0.9997684955596924, 
F1: 0.9950149599580262


# Saving the model to use it for predictions, heatmaps, etc.

In [38]:
save_model(model, 'vgg16_heavily_reduced')